In [1]:
cd "Trabajo 1"

[WinError 2] The system cannot find the file specified: 'Trabajo 1'
c:\Users\etien\OneDrive\Documentos\Ramos\Inteligencia de negocios\BI_SNN


In [185]:
import pandas as pd
import numpy as np

def iniW(nodes_hidden, x, y):
    size_input, _ = x.shape # 375, 5
    _, size_output = y.shape # 375, 1
    
    # Init. w1 y w2 
    w1 = np.random.random((nodes_hidden, size_input)) #dim -> (20x5)
    w2 = np.random.random((size_output, nodes_hidden)) #dim -> (1x20)
       
    return(w1,w2)

def forward(x,w1,w2):
    # Calcula la activación de los Nodos Ocultos
    z1 = np.dot(w1, x)
    a1 = act_sigmoid(z1)
    # Calcula la activación de los Nodos de Salida
    z2 = np.dot(w2, a1)
    a2 = act_sigmoid(z2)

    return a2

def grad_bp(Act, w1, w2, e):
    a2 = Act                # 1, 375
    z2 = deriva_sigmoid(a2) # 1, 375
    a1 = np.dot(w2.T, z2)   # 20, 375
    z1 = deriva_sigmoid(a1) # 20, 375
    x = np.dot(w1.T, z1)    # 5, 375
    # e = 1, 375
    # w1 = 20, 5     | w2 = 1, 20
    # delta1 = 20, 5 | delta2 = 1, 375
    # dCdW1 = 20, 5  | dCdW2 = 1, 20

    # Calcular gradiente capa salida
    delta2 = np.multiply(e, deriva_sigmoid(z2))
    dCdW2 = np.dot(delta2, a1.T)
    # Calcular gradiente capa oculta
    delta1 = np.multiply( np.dot(w2.T, delta2), deriva_sigmoid(z1) )
    dCdW1 = np.dot( delta1, x.T)
    
    return dCdW1, dCdW2

#Activation function
def act_sigmoid(z):
    return(1/(1+np.exp(-z)))   

# Derivate of the activation funciton
def deriva_sigmoid(a):
    return(a*(1-a))

def load_config():      
    param = np.genfromtxt("config.csv",delimiter=',',dtype=None)    
    par=[]    
    par.append(np.int16(param[0])) # Number of nodes
    par.append(np.int16(param[1])) # Max. Iterations
    par.append(np.float(param[2])) # Learn rate    
    return (par)
    
# Load data 
def load_data(fname):
    df = pd.read_csv(fname, header=None)   
    df = df.sample(frac=1).reset_index(drop=True) # Reordenar valores aleatoriamente.

    x, y = norm(df) # Normalizar

    return (x, y)

def save_w(w1,w2, fname_w, cost, fname_cost):    
    # Guardar pesos 
    np.savez_compressed(fname_w, w1, w2)
    # Guardar mse
    archivo = open(fname_cost, 'w')
    archivo.write('Iteracion,MSE\n') # header
    [ archivo.write(f'{i},{c}\n') for i, c in enumerate(cost, 1) ]
    archivo.close()

def norm(df):
    a = 0.01
    b = 0.99
    df_norm = pd.DataFrame()

    for col in df:
        min_ = df[col].min()
        max_ = df[col].max()

        df_norm[col] = (df[col] - min_) * (b - a) / (max_ - min_) + a
    
    x = df_norm.drop(labels=[5], axis=1)
    x = x.T
    y = df_norm.drop(labels=[i for i in range(5)],axis = 1)
    
    return (x.to_numpy(),y.to_numpy())

#Training of SNN
def train_snn(x,y,param):
    w1,w2 = iniW(param[0], x, y)  
    cost = []
    for iter in range(param[1]):
        #Step 1: Forward
        Act = forward(x,w1,w2) 
      
        #Step2: Backward
        w1, w2, mse = backward(Act, y, w1,w2,param[2]) 
        cost.append(mse)
        break
    return(w1,w2,cost)


In [187]:
def backward(Act, y, w1,w2, mu):
    # Calcular el error
    error = y.T - Act
    # Calcular el gradiente oculto y salida    
    dCdW = grad_bp(Act, w1, w2, error)
    # Actualizar los pesos
    w1, w2 = updW(w1, w2, mu, dCdW)
    
    mse = 0
    for e in error[0]:
        mse += e**2
    
    return w1, w2, mse 

# Update SNN's Weight 
def updW(w1, w2, mu=0.1, dCdW=(0,0)):    
    dCdW1, dCdW2 = dCdW
    # Actualizar pesos ocultos
    w1 = w1 - mu * dCdW1   
    # Actualizar pesos salida
    w2 = w2 - mu * dCdW2
    return w1, w2

# Beginning ...
def main():
    par_snn         = load_config()    
    xe,ye           = load_data('train.csv')     
    w1,w2, cost     = train_snn(xe,ye,par_snn)         
    save_w(w1,w2,'w_snn.npz',cost,'costo.csv')

main()

cost: 55.435432232263246


<ipython-input-185-a68e70e54df0>:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  par.append(np.float(param[2])) # Learn rate


In [ ]:
Xe = df_sh.drop(labels=[5], axis=1)
Xe.head(5)

,0,1,2,3,4
0,-37.258,-1.2665,0.97613,1.5622,0.50928
1,-47.914,-2.2269,1.72620,5.8415,0.50005
2,-48.384,-2.4117,2.38100,6.3807,0.50126
3,-37.604,-1.3042,0.97883,1.7130,0.49855
4,12.109,5.8281,-13.61900,-17.8160,1.64490


In [ ]:
Ye = df_sh[5]
Ye.head(5)

0    5.0609
1    5.3799
2    5.5279
3    5.0649
4    3.1889
Name: 5, dtype: float64

In [ ]:
Xe.to_numpy()

array([[-37.258  ,  -1.2665 ,   0.97613,   1.5622 ,   0.50928],
       [-47.914  ,  -2.2269 ,   1.7262 ,   5.8415 ,   0.50005],
       [-48.384  ,  -2.4117 ,   2.381  ,   6.3807 ,   0.50126],
       ...,
       [ -0.46898,   0.91531, -10.332  ,   4.5917 ,   0.33055],
       [ 14.89   ,   3.8299 ,  -7.0057 , -14.29   ,   0.41149],
       [-20.783  ,  -0.20817,  -1.4272 ,  -6.7446 ,   0.68574]])

In [ ]:
Ye.min()

1.6459

In [ ]:
Ye.max()

5.5959

In [ ]:
Xe.mean()

0   -8.585263
1    1.917864
2    2.113191
3   -0.360373
4   -0.029521
dtype: float64

In [ ]:
def normalize(df):
    a = 0.01
    b = 0.99
    df_norm = pd.DataFrame()

    for col in df:
        min_ = df[col].min()
        max_ = df[col].max()

        df_norm[col] = (df[col] - min_) * (b - a) / (max_ - min_) + a

    return df_norm

normalize(Xe).head()

,0,1,2,3,4
0,0.158503,0.199644,0.482998,0.626218,0.518994
1,0.077592,0.163670,0.487381,0.715058,0.517838
2,0.074024,0.156748,0.491208,0.726252,0.517990
3,0.155876,0.198231,0.483013,0.629348,0.517650
4,0.533344,0.465384,0.397703,0.223916,0.661250


In [ ]:
Xe

,0,1,2,3,4
0,-37.25800,-1.26650,0.97613,1.5622,0.50928
1,-47.91400,-2.22690,1.72620,5.8415,0.50005
2,-48.38400,-2.41170,2.38100,6.3807,0.50126
3,-37.60400,-1.30420,0.97883,1.7130,0.49855
4,12.10900,5.82810,-13.61900,-17.8160,1.64490
...,...,...,...,...,...
370,1.00230,6.10800,-16.02300,4.4621,-1.04070
371,-8.42550,0.87524,-1.41120,-5.2240,1.24650
372,-0.46898,0.91531,-10.33200,4.5917,0.33055
373,14.89000,3.82990,-7.00570,-14.2900,0.41149


In [ ]:
Ye

0      5.0609
1      5.3799
2      5.5279
3      5.0649
4      3.1889
        ...  
370    3.9109
371    4.3779
372    4.0539
373    2.1989
374    4.5859
Name: 5, Length: 375, dtype: float64

In [ ]:
nodes = 20
x = 5
w1 = [ [ np.random.random() for n in range(nodes) ] for i in range(x) ]
len(w1[0])

20

In [ ]:
w1 = np.random.random((nodes, x))
w2 = np.random.random((1, nodes)) #dim -> (1x20)

In [ ]:
def act_sigmoid(z):
    return(1/(1+np.exp(-z))) 

def deriva_sigmoid(a):
    return(a*(1-a))

In [ ]:
normalize(Xe).loc[0]

0    0.158503
1    0.199644
2    0.482998
3    0.626218
4    0.518994
Name: 0, dtype: float64

In [ ]:
np.dot(w1, normalize(Xe).loc[0]) # (20x5) * ()

array([1.02226303, 0.99370184, 0.67088017, 1.07444246, 1.12762387,
       1.14110654, 1.35783563, 1.12483606, 1.19064053, 1.03201658,
       1.06613168, 1.13949412, 0.5511861 , 1.02380804, 1.16529267,
       0.65247161, 0.67902343, 1.58134159, 1.19534153, 1.17252565])

In [ ]:
normalize(Xe).to_numpy().transpose().shape

(5, 375)

In [ ]:
np.dot(w1, normalize(Xe).to_numpy().transpose()).shape

(20, 375)

In [ ]:
normalize(df).drop(labels=[i for i in range(5)],axis = 1)

,5
0,0.441696
1,0.687813
2,0.717337
3,0.450132
4,0.656056
...,...
370,0.477919
371,0.801691
372,0.162582
373,0.796729


In [ ]:
def norm(df):
    a = 0.01
    b = 0.99
    df_norm = pd.DataFrame()

    for col in df:
        min_ = df[col].min()
        max_ = df[col].max()

        df_norm[col] = (df[col] - min_) * (b - a) / (max_ - min_) + a
    
    x = df_norm.drop(labels=[5], axis=1)
    y = df_norm.drop(labels=[i for i in range(5)],axis = 1)

    return (x.to_numpy(),y.to_numpy())

In [ ]:
Xe, Ye = norm(df)
Xe.T.shape

(5, 375)

In [ ]:
z1 = np.dot(w1, Xe.T )
z1

array([[1.80980121, 1.14632778, 1.01226885, ..., 1.5767197 , 1.07781434,
        1.20114475],
       [1.55882706, 1.1419205 , 0.97029919, ..., 1.44198233, 1.03098115,
        1.14707218],
       [1.24510067, 0.74245576, 0.6635935 , ..., 1.32717471, 0.72029434,
        0.79783178],
       ...,
       [2.65186715, 1.7949812 , 1.60050915, ..., 2.25937725, 1.649621  ,
        1.89403348],
       [2.15985143, 1.35580719, 1.22745146, ..., 2.00341228, 1.26548291,
        1.46986645],
       [1.84164907, 1.25228654, 1.09700172, ..., 1.55600444, 1.21815415,
        1.26466385]])

In [ ]:
z2 = np.dot(w2, act_sigmoid(z1))
Act = act_sigmoid(z2).T
Act

array([[0.99990372],
       [0.99975304],
       [0.99968047],
       [0.999912  ],
       [0.99964623],
       [0.99976979],
       [0.99974489],
       [0.99968867],
       [0.9997099 ],
       [0.99968861],
       [0.99969477],
       [0.99972371],
       [0.99970233],
       [0.99971148],
       [0.99967029],
       [0.99967931],
       [0.99968575],
       [0.9997813 ],
       [0.99969308],
       [0.99974289],
       [0.99993299],
       [0.99972559],
       [0.99966226],
       [0.99964952],
       [0.99974042],
       [0.99974363],
       [0.99986976],
       [0.99968503],
       [0.99972124],
       [0.99971805],
       [0.99969679],
       [0.99969658],
       [0.99978085],
       [0.99966924],
       [0.99993289],
       [0.99970404],
       [0.9996749 ],
       [0.99972943],
       [0.99967853],
       [0.9996861 ],
       [0.99991757],
       [0.99986028],
       [0.99967807],
       [0.99973041],
       [0.99966486],
       [0.99970112],
       [0.99988102],
       [0.999

In [ ]:
Ye

array([[0.4416962 ],
       [0.68781266],
       [0.71733671],
       [0.45013165],
       [0.6560557 ],
       [0.6962481 ],
       [0.64588354],
       [0.73842532],
       [0.79077468],
       [0.49354937],
       [0.77340759],
       [0.68533165],
       [0.81508861],
       [0.64042532],
       [0.96891139],
       [0.91829873],
       [0.77092658],
       [0.67764051],
       [0.78878987],
       [0.55036456],
       [0.45856709],
       [0.64017722],
       [0.98702278],
       [0.53696709],
       [0.37173165],
       [0.73916962],
       [0.01      ],
       [0.85825823],
       [0.76794937],
       [0.77117468],
       [0.83022278],
       [0.80268354],
       [0.54589873],
       [0.62504304],
       [0.15737215],
       [0.88703797],
       [0.82451646],
       [0.72924557],
       [0.94484557],
       [0.85726582],
       [0.34419241],
       [0.27918987],
       [0.73644051],
       [0.60767595],
       [0.66101772],
       [0.63099747],
       [0.21344304],
       [0.260

In [ ]:
e = Act - Ye
e.shape

(375, 1)

In [ ]:
deriva_sigmoid(z2)

array([[-76.27979654, -60.68445851, -56.72738771, -77.86140927,
        -55.20037249, -61.78622958, -60.17818762, -57.12064287,
        -58.1957657 , -57.117608  , -57.42086676, -58.94414761,
        -57.80238206, -58.27946149, -56.25460323, -56.67276478,
        -56.97952303, -62.59653671, -57.33689616, -60.05681886,
        -82.75192233, -59.04937094, -55.89365429, -55.33969683,
        -59.90854622, -60.10187026, -71.0850306 , -56.94483071,
        -58.80727078, -58.63239941, -57.52210879, -57.51156024,
        -62.56393544, -56.20688195, -82.7263834 , -57.89046876,
        -56.46693522, -59.26648787, -56.63624025, -56.99636025,
        -79.02254634, -69.90300337, -56.61469116, -59.32300191,
        -56.00947785, -57.74074808, -72.62063105, -75.96912368,
        -79.47672373, -56.64694494, -57.27622226, -62.165099  ,
        -56.27969018, -55.91167143, -65.72293996, -59.42683221,
        -58.07332225, -64.17495622, -56.90257179, -59.57303177,
        -61.01580761, -60.26294168, -62.

In [ ]:
np.arange(9.0).reshape((3, 3)) 

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [165]:
sum(np.arange(9.0))

36.0

In [ ]:
np.arange(9.0).reshape((3, 3)) * np.arange(3.0)

array([[ 0.,  1.,  4.],
       [ 0.,  4., 10.],
       [ 0.,  7., 16.]])